## 2 Private Training
Student Name & Number =Sara Rezanezhad / 99101643
Student Name & Number =Kimia Fakheri / *******


In [1]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.utils import resample
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from matplotlib import pyplot


In [15]:

def load_mnist():
    from keras.datasets import fashion_mnist

    # load dataset
    (trainX, trainY), (testX, testY) = fashion_mnist.load_data()
    # reshape dataset to have a single channel
    trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
    testX = testX.reshape((testX.shape[0], 28, 28, 1))
    # one hot encode target values
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)

    return (trainX, trainY), (testX, testY)
# Number of target models
NUM_TARGET = 1
# Number of shadow models
NUM_SHADOW = 2
VERBOSE = 3

"""
    Samples the training and testing data to create multiple smaller datasets.

    Args:
        train_data (Tuple[np.ndarray, np.ndarray]): Original training data (X_train, y_train).
        test_data (Tuple[np.ndarray, np.ndarray]): Original testing data (X_test, y_test).
        num_sets (int): Number of smaller datasets to create.

    Returns:
        Tuple[Tuple[list, list], Tuple[list, list]]:
            - Resampled training data ([X_train1, X_train2, ...], [y_train1, y_train2, ...])
            - Resampled testing data ([X_test1, X_test2, ...], [y_test1, y_test2, ...])
    """
def sample_data(train_data, test_data, num_sets):
    (x_train, y_train), (x_test, y_test) = train_data, test_data
    x_train2, y_train2 = [], []
    x_test2, y_test2 = [], []
    for i in range(num_sets):
        x_temp, y_temp = resample(x_train, y_train, n_samples=3000, random_state=0)
        x_train2.append(x_temp)
        y_train2.append(y_temp)
        x_temp, y_temp = resample(x_test, y_test, n_samples=2000, random_state=0)
        x_test2.append(x_temp)
        y_test2.append(y_temp)
    return (x_train2, y_train2), (x_test2, y_test2)

    """
    Generates a dataset for the attack model by extracting predictions from the target and shadow models.

    Args:
        models (list): List of trained target and shadow models.
        train_data (Tuple[np.ndarray, np.ndarray]): Original training data (X_train, y_train).
        test_data (Tuple[np.ndarray, np.ndarray]): Original testing data (X_test, y_test).
        num_models (int): Number of target and shadow models.
        data_size (int): Size of the attack dataset.

    Returns:
        Tuple[list, list]:
            - X_data: List of model predictions, where each inner list represents a class.
            - y_data: List of labels, where 1 represents a target model and 0 represents a shadow model.
    """
def attack_dataset(models, train_data, test_data, num_models, data_size):
    # generate dataset for the attack model
    (x_train, y_train), (x_test, y_test) = train_data, test_data
    # set number of classes for the attack model
    num_classes = 10
    x_data, y_data = [[] for _ in range(num_classes)], [[] for _ in range(num_classes)]
    for i in range(num_models):

        x_temp, y_temp = resample(x_train[i], y_train[i], n_samples=data_size, random_state=0)
        for j in range(data_size):
            y_idx = np.argmax(y_temp[j])
            x_data[y_idx].append(models[i].predict(x_temp[j:j+1], verbose=0)[0])
            y_data[y_idx].append(1)
        x_temp, y_temp = resample(x_test[i], y_test[i], n_samples=data_size, random_state=0)
        for j in range(data_size):
            y_idx = np.argmax(y_temp[j])
            x_data[y_idx].append(models[i].predict(x_temp[j:j+1], verbose=0)[0])
            y_data[y_idx].append(0)

    return x_data, y_data

    """
    Defines a Fully Connected Neural Network (FCNN) model for the Fashion-MNIST classification task.

    Returns:
        tf.keras.Model: The FCNN model.
    """
def FCNN():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),  # Add dropout with 50% rate
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

def get_trained_keras_models(keras_model, train_data, test_data, num_models):
    (x_train, y_train), (x_test, y_test) = train_data, test_data
    models = []
    for i in range(num_models):
        models.append(tf.keras.models.clone_model(keras_model))
        rms = tf.keras.optimizers.RMSprop(learning_rate=0.001)
        models[i].compile(loss='categorical_crossentropy', optimizer=rms, metrics=['accuracy'])
        models[i].fit(x_train[i], y_train[i], batch_size=32, epochs=5, verbose=VERBOSE, shuffle=True)
        score = models[i].evaluate(x_test[i], y_test[i], verbose=VERBOSE)
        print('\n', 'Model ', i, ' test accuracy:', score[1])
    return models

def get_trained_svm_models(train_data, test_data, num_models=1):
    from sklearn import svm
    (x_train, y_train), (x_test, y_test) = train_data, test_data
    models = []
    for i in range(num_models):
        print('Training svm model : ', i)
        models.append(svm.SVC(gamma='scale', kernel='linear', verbose=VERBOSE))
        models[i].fit(x_train[i], y_train[i])
        score = models[i].score(x_test[i], y_test[i])
        print('SVM model ', i, 'score : ', score)
    return models





In [17]:

    # load the pre-shuffled train and test data
    (x_train, y_train), (x_test, y_test) = load_mnist()

    # split the data for each model
    target_train = (x_train[:3000*NUM_TARGET], y_train[:3000*NUM_TARGET])
    target_test = (x_test[:2000*NUM_TARGET], y_test[:2000*NUM_TARGET])
    target_train_data, target_test_data = sample_data(target_train, target_test, NUM_TARGET)

    shadow_train = (x_train[3000*NUM_TARGET:], y_train[3000*NUM_TARGET:])
    shadow_test = (x_test[2000*NUM_TARGET:], y_test[2000*NUM_TARGET:])
    shadow_train_data, shadow_test_data = sample_data(shadow_train, shadow_test, NUM_SHADOW)

    CNN_model = FCNN()

    # Q5
    target_models = get_trained_keras_models(CNN_model, target_train_data, target_test_data, NUM_TARGET)
    # Q6
    shadow_models = get_trained_keras_models(CNN_model, shadow_train_data, shadow_test_data, NUM_SHADOW)

    # get train data for the attack model
    attack_train = attack_dataset(shadow_models, shadow_train_data, shadow_test_data, NUM_SHADOW, 2000)
    # Q7
    attack_test = attack_dataset(target_models, target_train_data, target_test_data, NUM_TARGET, 2000)

    # training the attack model
    attack_model = get_trained_svm_models(attack_train, attack_test)

4422102/4422102 [==============================] - 1s 0us/step
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5

 Model  0  test accuracy: 0.5245000123977661
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5

 Model  0  test accuracy: 0.5715000033378601
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5

 Model  1  test accuracy: 0.6065000295639038
Training svm model :  0
[LibSVM]SVM model  0 score :  0.5
